In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

TypedDict

In [2]:
from typing import Dict, TypedDict

class Person(TypedDict):
    name: str
    age: int    # int 타입으로 넣기
    job: str

In [3]:
hong: Person = {'name': '홍길동', 'age': 20, 'job': '프로그래머'}
hong

{'name': '홍길동', 'age': 20, 'job': '프로그래머'}

In [4]:
hong['name']

'홍길동'

In [5]:
hong['age']

20

Annotated

In [6]:
from typing import Annotated

name: Annotated[str, "사용자 이름"]
age: Annotated[int, "사용자 나이 (0-150)"]

In [7]:
from typing import Annotated, List
from pydantic import Field, BaseModel, ValidationError

class Employee(BaseModel):
    id: Annotated[int, Field(..., description="직원 ID")]
    name: Annotated[str, Field(..., min_length=3, max_length=50, description="이름")]
    age: Annotated[int, Field(gt=18, lt=65, description="나이 (19-99세)")]
    salary: Annotated[float, Field(gt=0, lt=10000, description="연봉 (단위: 만원, 최대 10억)")]
    skills: Annotated[List[str], Field(min_items=1, max_items=10, description="보유 기술 (1-10개)")]

In [8]:
# 유효한 데이터로 인스턴스 생성
try:
    valid_employee = Employee(
        id=1, 
        name="홍길동", 
        age=30, 
        salary=8000, 
        skills=["Python", "LangChain"]
    )

    print("유효한 직원 데이터:", valid_employee)    
except ValidationError as e:
    print("유효성 검사 오류:", e)

유효한 직원 데이터: id=1 name='홍길동' age=30 salary=8000.0 skills=['Python', 'LangChain']


In [9]:
# 유효하지 않은 데이터로 인스턴스 생성 시도
try:
    invalid_employee = Employee(
        name="윤정",        # 이름이 너무 짧음
        age=9,              # 나이가 범위를 벗어남
        salary=20000,       # 급여가 범위를 벗어남
        skills="Python",    # 리스트가 아님
    )
except ValidationError as e:
    print("유효성 검사 오류:")
    for error in e.errors():
        print(f"- {error['loc'][0]}: {error['msg']}")

유효성 검사 오류:
- id: Field required
- name: String should have at least 3 characters
- age: Input should be greater than 18
- salary: Input should be less than 10000
- skills: Input should be a valid list


### LangGraph에서의 사용(add_messages)

`add_messages` 는 LangGraph 에서 메시지를 리스트에 추가하는 함수입니다.

In [10]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages

class MyData(TypedDict):
    messages: Annotated[list, add_messages]

### add_messages

`messages` 키는 `add_messages` 리듀서 함수로 주석이 달려 있으며, 이는 LangGraph에게 기존 목록에 새 메시지를 추가하도록 지시합니다. <br>

주석이 없는 상태 키는 각 업데이트에 의해 덮어쓰여져 가장 최근의 값이 저장됩니다. <br>

`add_messages` 함수는 2개의 인자(left, right)를 받으며 좌, 우 메시지를 병합하는 방식으로 동작합니다.

In [11]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

# 기본 사용 예시
msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [AIMessage(content="반갑습니다~",    id="2")]

result1 = add_messages(msgs1, msgs2)
print(result1)

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='1'), AIMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='2')]
